In [73]:
import pandas as pd
from FedTools import FederalReserveMins, MonetaryPolicyCommittee
import datetime
import spacy
from tqdm import tqdm

In [56]:
speeches = pd.read_csv('data/fed_speeches_1996_2020.csv').dropna(subset=['date', 'text'])
speeches['date'] = speeches['date'].apply(lambda x: datetime.datetime.strptime(str(int(x)), '%Y%m%d'))
speeches['type'] = 'speech'

In [14]:
statements = MonetaryPolicyCommittee().find_statements()
statements['type'] = 'fomc_statement'

Constructing links between 1994 and 2023
Extracting the past 195 FOMC Statements.
Retrieving articles.
...................................................................................................................................................................................................

In [60]:
## let's stick with statements and speeches, since the minutes contain a lot of extra nonsense
df = pd.concat([
    statements.reset_index().rename({"index": "date", "FOMC_Statements": 'text'}, axis=1),
    speeches[['date', 'text', 'type']]
])

In [70]:
model = spacy.load('en_core_web_sm')

In [82]:
def prepro(text, model):
    text = model(text.lower().strip())
    
    clean_text = []
    for word in text:
        # remove stopwords and words with odd punctuation
        if (not word.is_stop) and (word.pos_ not in ['SYM', 'NUM', 'ADP', 'AUX', 'PROPN']) and (not word.is_punct):
            clean_text.append(word.lemma_.strip())
            
    return ' '.join(clean_text)

In [83]:
cleans = []
for t in tqdm(df['text']):
    cleans.append(prepro(t, model))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1650/1650 [10:12<00:00,  2.69it/s]


In [86]:
df['clean_text'] = cleans
df.sort_values('date').reset_index().drop(columns=['index']).to_parquet('data/combined_clean.parquet')

In [93]:
model('hello').vector.shape

(96,)